In [1]:
!pip install mysql-connector-python

In [2]:
import mysql.connector  

connection = mysql.connector.connect(
    host="127.0.0.1",     
    user="root",         
    password="2011DongBaLiem@",
    database="preday48")
cursor = connection.cursor()

In [3]:
import pandas as pd

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS customers(
    id INT PRIMARY KEY,
    name VARCHAR(100),
    city VARCHAR(100)
) ENGINE=InnoDB;
""")

In [6]:
cursor.execute("""
INSERT INTO customers VALUES
    (1, 'Alice', 'Hanoi'),
    (2, 'Bob', 'Ho Chi Minh'),
    (3, 'Charlie', 'Hanoi')
;
""") 
connection.commit()

In [7]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS orders(
    id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total INT
)ENGINE=InnoDB;
""")

In [8]:
cursor.execute("""
INSERT INTO orders VALUES
    (1, 1, '2024-01-10', 500),
    (2, 1, '2024-03-01', 800),
    (3, 2, '2024-02-20', 1200),
    (4, 3, '2024-01-15', 200)
;
""")
connection.commit()

In [9]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS products(
    id INT PRIMARY KEY,
    name VARCHAR(100),
    price INT
)ENGINE=InnoDB;
""")

In [10]:
cursor.execute("""
INSERT INTO products VALUES
    (1, 'Laptop', 1500),
    (2, 'Mouse', 50),
    (3, 'Keyboard', 100),
    (4, 'Monitor', 300)
;
""")
connection.commit()

In [11]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS order_items(
    id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT
)ENGINE=InnoDB;
""")

In [12]:
cursor.execute("""
INSERT INTO order_items VALUES
    (1, 1, 2, 2),
    (2, 1, 3, 1),
    (3, 2, 1, 1),
    (4, 2, 4, 1),
    (5, 3, 1, 1)
;
""")
connection.commit()

In [13]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees(
    id INT PRIMARY KEY,
    name VARCHAR(100),
    department VARCHAR(100)
)ENGINE=InnoDB;
""")

In [14]:
cursor.execute("""
INSERT INTO employees VALUES
    (1, 'David', 'Sales'),
    (2, 'Emma', 'Support'),
    (3, 'Frank', 'Sales')
;
""")
connection.commit()

In [15]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS order_assignments(
    id INT PRIMARY KEY,
    order_id INT,
    employee_id INT
)ENGINE=InnoDB;
""")

In [16]:
cursor.execute("""
INSERT INTO order_assignments VALUES
    (1, 1, 1),
    (2, 2, 3),
    (3, 3, 2)
;
""") 
connection.commit()

In [17]:
# Tìm tất cả khách hàng đã từng mua đơn hàng chứa sản phẩm có giá lớn hơn 1000
query="""
SELECT DISTINCT c.name
FROM customers c
JOIN orders o ON c.id= o.customer_id
WHERE o.id IN(
    SELECT oi.order_id
    FROM order_items oi
    JOIN products p ON oi.product_id = p.id
    WHERE p.price > 1000)
;
"""
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27364\3277248752.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name
0,Alice
1,Bob


In [18]:
# Liệt kê tên từng khách hàng và tổng giá trị tất cả đơn hàng mà họ đã đặt
query="""
SELECT c.name,
    (SELECT SUM(total)
    FROM orders o
    WHERE o.customer_id= c.id) AS total_spent
FROM customers c
;
"""
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27364\2387988103.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name,total_spent
0,Alice,1300.0
1,Bob,1200.0
2,Charlie,200.0


In [19]:
# Hiển thị top 2 khách hàng có tổng chi tiêu cao nhất (dựa trên cột total của đơn hàng)
query="""
SELECT name, total_spent
FROM (
    SELECT c.name, sum(o.total) AS total_spent
    FROM customers c
    JOIN orders o ON c.id = o.customer_id
    GROUP BY c.name
) AS spending
ORDER BY total_spent DESC
LIMIT 2
;
"""
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27364\426378739.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name,total_spent
0,Alice,1300.0
1,Bob,1200.0


In [22]:
# Tìm tên nhân viên thuộc phòng Sales đã từng xử lý đơn hàng có chứa sản phẩm tên "Monitor"
query="""
SELECT DISTINCT e.name
FROM employees e
WHERE e.department = 'Sales'
AND EXISTS(
    SELECT 1
    FROM order_assignments oa 
    JOIN order_items oi ON oi.order_id = oa.order_id
    JOIN products p ON p.id = oi.product_id
    WHERE p.name = 'Monitor'
    AND oa.employee_id = e.id )
;
"""
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27364\3368298243.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name
0,Frank


In [6]:
# Tìm các khách hàng có tổng chi tiêu cao hơn mức trung bình của tất cả khách hàng
query="""
SELECT c.name, SUM(o.total) AS total_spent
FROM customers c
JOIN orders o ON c.id = o.customer_id
GROUP BY c.name
HAVING SUM(o.total) > (
    SELECT AVG(customer_total)
    FROM (
        SELECT customer_id, SUM(total) AS customer_total
        FROM orders
        GROUP BY customer_id
    ) AS avg_spent
);
"""
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3460\2450876850.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name,total_spent
0,Alice,1300.0
1,Bob,1200.0


In [10]:
# Tạo bảng tạm vip_customers gồm những khách hàng có tổng giá trị đơn hàng lớn hơn 1000.
# Sau đó, hiển thị tên và tổng chi tiêu của các khách hàng trong bảng tạm này.

query="""
CREATE TEMPORARY TABLE IF NOT EXISTS vip_customers AS
SELECT customer_id, SUM(total) AS total_spent
FROM orders 
GROUP BY customer_id
HAVING SUM(total) > 1000
;
"""
cursor.execute(query)

In [13]:
query="""
SELECT c.name, v.total_spent
FROM customers c
JOIN vip_customers v ON c.id = v.customer_id
;"""
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3460\2234078328.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name,total_spent
0,Alice,1300.0
1,Bob,1200.0


In [14]:
# Tạo bảng tạm popular_products gồm những sản phẩm được bán ra với tổng số lượng từ 2 trở lên
# Hiển thị tên và tổng số lượng bán ra của các sản phẩm này
query="""
CREATE TEMPORARY TABLE IF NOT EXISTS popular_products AS
SELECT product_id, SUM(quantity) AS total_quantity
FROM order_items
GROUP BY product_id
HAVING SUM(quantity) >=2 
;"""
cursor.execute(query)

In [15]:
query="""
SELECT p.name, pp.total_quantity
FROM products p
JOIN popular_products pp ON pp.product_id = p.id
;"""
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3460\784532834.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name,total_quantity
0,Mouse,2.0
1,Laptop,2.0


In [23]:
# Liệt kê tên các khách hàng có đơn hàng được xử lý bởi nhân viên có tổng doanh số lớn hơn 1000
query="""SELECT c.name
FROM customers c
WHERE EXISTS (
    SELECT 1
    FROM orders o
    JOIN order_assignments oa ON o.id = oa.order_id
    WHERE o.customer_id = c.id
    AND oa.employee_id IN (
        SELECT oa.employee_id
        FROM order_assignments oa
        JOIN orders o ON oa.order_id = o.id
        GROUP BY oa.employee_id
        HAVING SUM(o.total) > 1000
)
);"""
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3460\3753537694.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name
0,Bob


In [17]:
query="""
SELECT DISTINCT c.name
FROM customers c
JOIN orders o ON o.customer_id = c.id
JOIN order_assignments oa ON oa.order_id = o.id
WHERE oa.employee_id IN (
    SELECT employee_id
    FROM (
        SELECT e.id AS employee_id, SUM(o.total) AS total_sales
        FROM employees e
        JOIN order_assignments oa ON e.id = oa.employee_id
        JOIN orders o ON o.id = oa.order_id
        GROUP BY e.id
        HAVING SUM(o.total) > 1000
    ) AS top_sellers
);"""
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3460\3687484891.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name
0,Bob


In [18]:
query="""
SELECT DISTINCT c.name
FROM customers c
JOIN orders o ON c.id = o.customer_id
JOIN order_assignments oa ON o.id = oa.order_id
WHERE oa.employee_id IN(
    SELECT employee_id
    FROM (
        SELECT e.id AS employee_id, SUM(o.total) AS total_sales
        FROM employees e
        JOIN order_assignments oa ON oa.employee_id = e.id
        JOIN orders o ON o.id = oa.order_id
        GROUP BY e.id
        HAVING SUM(o.total) > 1000
    ) AS top_sellers


);"""
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3460\3021193990.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name
0,Bob


In [20]:
# Tạo bảng tạm high_value_orders gồm những đơn hàng có tổng tiền > 800
query="""
CREATE TEMPORARY TABLE IF NOT EXISTS high_value_orders AS
SELECT *
FROM orders 
WHERE total >800
;"""
cursor.execute(query)

# Hiển thị tên khách hàng của các đơn hàng đó
query="""
SELECT c.name, hvo.total
FROM high_value_orders hvo
JOIN customers c ON c.id = hvo.customer_id
; """
connection.commit()
data=pd.read_sql_query(query,connection)
data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3460\971696740.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,name,total
0,Bob,1200


In [21]:
# Tính tổng giá trị thực tế của từng đơn hàng bằng cách lấy price × quantity theo từng dòng sản phẩm
query="""
SELECT oi.order_id, SUM(p.price*oi.quantity) AS total_value_oi
FROM order_items oi
JOIN products p ON p.id = oi.product_id
GROUP BY oi.order_id
; """
connection.commit()
data=pd.read_sql_query(query,connection)
data



C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3460\1222332751.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data=pd.read_sql_query(query,connection)


,order_id,total_value_oi
0,1,200.0
1,2,1800.0
2,3,1500.0
